In [5]:
#imports
import torch
import copy
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import gym
from collections import deque, namedtuple
import os
import pandas as pd
import matplotlib.pyplot as plt
from multiprocessing import Manager, Lock, Process

In [2]:
class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

In [3]:
Transition = namedtuple('Transition', 
                        ('observation', 'action', 'next_observation', 'reward', 'done'))

class ReplayMemoryMP(object):

    def __init__(self, capacity):
        manager = Manager()
        self.memory = manager.list()
        self.capacity = capacity
        self.lock = Lock()

    def push(self, *args):
        """save a transtion"""
        with self.lock:
            if len(self.memory) >= self.capacity:
                self.memory.pop(0)
                self.memory.append(Transition(*args))
        #print(self.memory)

    def sample(self, batch_size):
        with self.lock:
            return random.sample(list(self.memory), batch_size)
    
    def __len__(self):
        return len(self.memory)

In [4]:
def pick_action(epsilon, policy_net, env, obs):
    p = random.uniform(0,1)
    if p < epsilon:
        action = env.action_space.sample()
    else:
        if isinstance(obs, np.ndarray):
            obs = torch.tensor(obs, dtype=torch.float32)

        if obs.ndim == 1:
            obs = obs.unsqueeze(0)

        with torch.no_grad():
            q_values = policy_net(obs)
            action = torch.argmax(q_values).item()
    return action

def error(current_q, td_target, error_type):
    if error_type == "mse":
        #error = (current_q - td_target) ** 2
        compute_loss = nn.MSELoss()
        error = compute_loss(current_q, td_target)
    
    return error

In [ ]:
def sample_batch(replay_buffer, batch_size):
    """Return tensors for observations, actions, rewards, next_observations, dones"""
    transitions = replay_buffer.sample(batch_size)
    batch = Transition(*zip(*transitions))

    obs = torch.stack([torch.tensor(o, dtype=torch.float32) for o in batch.observation])
    next_obs = torch.stack([torch.tensor(o, dtype=torch.float32) for o in batch.next_observation])
    a = torch.tensor(batch.action, dtype=torch.int64).unsqueeze(1)
    r = torch.tensor(batch.reward, dtype=torch.float32).unsqueeze(1)
    done = torch.tensor(batch.done, dtype=torch.float32).unsqueeze(1)
    
    return obs, a, r, next_obs, done

In [ ]:
class MP_agent():
    def __init__(self, id, replay_buffer, policy_net):
        self.id = id
        self.replay_buffer = replay_buffer
        self.policy_net = policy_net

In [ ]:
def MP_DQN(learning_rate, gamma, episodes, target_update, epsilon, capacity, batch_size, n_agents):

    # Initialize the policy network and optimizer
    env = gym.make('CartPole-v1')
    observation, _ = env.reset()
    policy_net = QNetwork(state_dim=4, action_dim=2)
    target_net = QNetwork(state_dim=4, action_dim=2)
    target_net.load_state_dict(policy_net.state_dict())
    optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate)
    plot_avg_rewards = []

    replay_buffer = ReplayMemoryMP(capacity=capacity) #initialize buffer, its shared between all agents
    step_count = 0 #global step count
    total_reward = 0

    for i in range(n_agents):
        p = Process(target=worker_fn, args=(i, replay_buffer))
        p.start()
    
    # Training loop
    for episode in range(episodes):
        observation, _ = env.reset() #observation for first action
        observation = torch.tensor(observation, dtype=torch.float32)
        terminated = False
        truncated = False 
        #step_count = 0

          
        while not terminated and not truncated:

            #perform an action, get env response
            action = pick_action(epsilon, policy_net, env, observation)
            next_observation, reward, terminated, truncated, __ = env.step(action)
            next_observation = torch.tensor(next_observation, dtype=torch.float32) 
            total_reward += reward

            #add new data to buffer
            replay_buffer.push(observation, action, next_observation, reward, terminated) 


            #do random steps until buffer has at least reached the batch_size
            if len(replay_buffer) < batch_size:
                step_count += 1
                continue 

            obs, a , r, obs_next, done = sample_batch(replay_buffer, batch_size)


            #update target network
            if step_count % target_update == 0:
                target_net.load_state_dict(policy_net.state_dict())

            step_count += 1
        
        
            next_q_values = target_net(obs_next)
            next_q_max = next_q_values.max(dim=1, keepdim=True)[0].detach()
            td_target = r + gamma * (1-done) * next_q_max

            current_q = policy_net(obs).gather(1, a)

            loss = error(current_q, td_target, "mse")
            
            #to see of weights are updating later
            #old_weights = [p.clone() for p in policy_net.parameters()]


            #update weights of the NN/policy
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            #tests if td_target is detatched - SHOULD NOT PRINT ANYTHING if it's implemented correctly
            for name, param in target_net.named_parameters():
                if param.grad is not None:
                    print(f"{name} has gradient!")

            #check if weights are updating
            # weight_diffs = [ (p - old_p).abs().mean().item() for p, old_p in zip(policy_net.parameters(), old_weights) ]
            # print("Average weight change per param:", weight_diffs)


            #update observation and decay epsilon for the next step
            observation = next_observation
            epsilon = max(0.01, 0.995 * epsilon) # decay epsilon


        if episode % 25 == 0:
            # calculate the avg rewards of the last 25 steps
            average = total_reward / 25
            plot_avg_rewards.append(average)
            #print(f'epsilon={epsilon:.3f}, total_reward={total_reward}')
            total_reward = 0
        
        if episode % 100 == 0:
            print(f'Episode {episode}/{episodes}')

    
    env.close()
    
    return plot_avg_rewards, policy_net


In [ ]:
def plot_results(learning_rate, gamma, episode, result):

    result = np.array(result)
    
    y = [25*i for i in range(len(result))]
    y_axis = np.array(y)

    #plot graph
    plt.plot(y_axis, result)
    plt.xlabel('Number of Episodes')
    plt.ylabel('Reward')
    plt.title('Average rewards, Learning Rate: {}, Gamma: {}, Episode: {}'.format(learning_rate, gamma, episode))
    plt.show()

    pass

In [ ]:
learning_rate = 0.005
gamma = 0.99
episodes = 500
target_update = 70
epsilon = 1
capacity = 10000
batch_size = 32
plot_avg_rewards, baseline_network = baseline_DQN(learning_rate, gamma, episodes, target_update, epsilon, capacity, batch_size)

plot_results(learning_rate, gamma, episodes, plot_avg_rewards)